In [1]:
import pandas as pd

In [3]:
## Подгрузка чернового варианта базы данных

data = pd.read_csv('105_2013_2019_features_v3_2.csv')

C:\Users\hamri\AppData\Local\Temp\ipykernel_15628\1907425286.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('105_2013_2019_features_v3_2.csv')


In [4]:
## Названия всех столбцов таблицы

data.columns

Index(['ID', 'custom_case_judge_id', 'Субъект РФ', 'Наименование суда',
       'case_judge', 'Номер дела (материала)', 'Тип документа', 'Статья УК РФ',
       'Дата поступления', 'Дата решения', 'ФИО', 'Результат', 'text',
       'Источник', 'n_offenders', 'case_date_upload', 'case_judge_id', 'year',
       'Судья', 'alcohol', 'alcohol_phrase', 'soft', 'soft_phrase', 'hard',
       'hard_phrase', 'prison_type', 'prison_type_phrase', 'gender_offender',
       'gender_offender_phrase', 'gender_victim', 'gender_victim_phrase',
       'sentence_period_phrase', 'sentence_period_years',
       'sentence_period_months', 'sentence_period_years_int',
       'sentence_period_months_int', 'text_prep', 'recidivism',
       'recidivism_phrase', 'guilty_plea', 'guilty_plea_phrase',
       'gender_judge', 'weapon', 'weapon_phrase', 'weapon_knife', 'weapon_gun',
       'weapon_hands', 'weapon_legs', 'weapon_other', 'weapon_not_defined',
       'weapon_not_used', 'soft_2', 'soft_2_phrase', 'hard_2', 'h

In [35]:
## Отбор релевантных для исследования признаков

data_rel = data[[
    'ID', 'Дата поступления', 'Дата решения', 'text', 'ФИО', 'gender_judge',
    'gender_victim', 'soft', 'hard', 'recidivism', 'guilty_plea', 'year', 
    'sentence_period_months_int', 'sentence_period_years_int'
]].copy()

In [51]:
## Подгрузка финальной базы данных  

data_reg = pd.read_csv('regression_data.csv')

In [52]:
## Названия всех столбцов финальной базы данных 

data_reg.columns

Index(['Unnamed: 0', 'ID', 'gender_victim', 'gender_judge', 'gender_offender',
       'recidivism_type', 'guilty_plea', 'soft_binary', 'hard_binary',
       'rel_type', 'year', 'children', 'sentence_period_whole', 'region',
       'Наименование суда'],
      dtype='object')

In [43]:
## Отбор наблюдений, имеющихся в обеих таблицах, по идентификатору уголовного дела

data_fast = data_rel.merge(data_reg, on='ID').copy()

In [132]:
## Названия всех столбцов обобщенной таблицы

data.columns

Index(['ID', 'custom_case_judge_id', 'Субъект РФ', 'Наименование суда',
       'case_judge', 'Номер дела (материала)', 'Тип документа', 'Статья УК РФ',
       'Дата поступления', 'Дата решения', 'ФИО', 'Результат', 'text',
       'Источник', 'n_offenders', 'case_date_upload', 'case_judge_id', 'year',
       'Судья', 'alcohol', 'alcohol_phrase', 'soft', 'soft_phrase', 'hard',
       'hard_phrase', 'prison_type', 'prison_type_phrase', 'gender_offender',
       'gender_offender_phrase', 'gender_victim', 'gender_victim_phrase',
       'sentence_period_phrase', 'sentence_period_years',
       'sentence_period_months', 'sentence_period_years_int',
       'sentence_period_months_int', 'text_prep', 'recidivism',
       'recidivism_phrase', 'guilty_plea', 'guilty_plea_phrase',
       'gender_judge', 'weapon', 'weapon_phrase', 'weapon_knife', 'weapon_gun',
       'weapon_hands', 'weapon_legs', 'weapon_other', 'weapon_not_defined',
       'weapon_not_used', 'soft_2', 'soft_2_phrase', 'hard_2', 'h

In [105]:
## Ключевая фильтрация уголовных дел по следующим параметрам: 
## 1) Год вынесения приговора -- 2017, 2018 или 2019
## 2) Пол жертвы -- женский
## 3) Пол убийцы -- мужской
## 4) Обвиняемый и потерпевшая являлись интимными партнерами 

data_fast_2019_sur = data_fast[
    (data_fast['year_x'].isin([2017, 2018, 2019])) &\
    (data_fast['gender_victim_x'] == 'женский') &\
    (data_fast['gender_offender'] == 'мужской') &\
    (data_fast['rel_type'] == '1 Убит партнер')
]['ID']

In [107]:
## Сохранение идентификаторов уголовного дела для дальнейшего поиска релевантных постов в ВК

data_fast_2019_sur.to_csv('fast_women_cases_id_2018_diplom.csv')

In [59]:
## Датасет с обучением модели

In [90]:
## Отбор наблюдений, которые есть в изначальной версии базы данных, 
## но нет в финальной 
## Для этого ставится тип объединения left

data_rel_all = data_rel.merge(data_reg, how='left', on='ID').copy()

In [100]:
## Отбор только тех уголовных дел, которых нет в финальной базе данных
## Фильтром стал признак наличия несовершеннолетних детей у обвиняемого (children)
## Соответственно, были отобраны только те наблюдения, где не нашлось данных в этой колонке,
## что характеризует наблюдения первоначальной базы данных
## Также была проведена фильтрация по году 

data_rell_all_2 = data_rel_all[
    (~data_rel_all['children'].notnull()) &\
    (data_rel_all['year_x'].isin([2017, 2018, 2019]))
].dropna(subset=['Дата решения']).copy()

In [104]:
## Для отобранных наблюдений необходимо выбрать идентификатор уголовного дела 
## для дальнейшего объединения с признаками, а также ФИО, текст приговора,  
## даты поступления уголовного дела в суд и вынесения решения по делу 

data_rell_all_2[[
    'ID', 
    'ФИО', 
    'text', 
    'Дата поступления', 
    'Дата решения'
]].to_csv('data_for_model_diplom.csv')